In [35]:
%serialconnect

[leftinbuffer] ['ets Jun  8 2016 00:22:57']
[leftinbuffer] ['']
[leftinbuffer] ['rst:0x1 (POWERON_RESET),boot:0x17 (SPI_FAST_FLASH_BOOT)']
[leftinbuffer] ['configsip: 0, SPIWP:0xee']
[leftinbuffer] ['clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00']
[leftinbuffer] ['mode:DIO, clock div:2']
[leftinbuffer] ['load:0x3fff0018,len:4']
[leftinbuffer] ['load:0x3fff001c,len:4332']
[leftinbuffer] ['load:0x40078000,len:0']
[leftinbuffer] ['load:0x40078000,len:10992']
[leftinbuffer] ['entry 0x4007a6c4']
[leftinbuffer] ['\x1bW (64) rtc_clk: Potentially bogus XTAL frequency: 35 MHz, guessing 40 MHz\x1b']
[leftinbuffer] ['Scanning wifis: BV6000']
[leftinbuffer] ['\x1bI (3032) network: event 1\x1b']
[leftinbuffer] ['Connecting to strongest known wifi signal BV6000']
attempt to exit past mode
[\r\x03\x02] b'Traceback (most recent call last):\r\n  File "main.py", line 1, in <module>\r\n  File "connectwifi.py", line 18, in <module>\r\nKeyboardInterrupt: \r\nMicroPython v1.9.2-443

In [3]:
%sendtofile --source i2cmodule.py

Sent 2 lines (72 bytes).


In [5]:
import os
print(os.listdir())

['boot.py', 'aaa', 'bbb.txt', 'ddd', 'dd', 'hhh.txt', 'i2cmodule.py']


In [36]:
from i2cmodule import i2c


In [37]:
print(0x77, (i2c.scan()))

119 [119]


In [39]:
import ustruct, time

C3, C4, C6, SENST1, OFFT1, Tref = 0,0,0,0,0,0
def setupms5611():
    global C3, C4, C6, SENST1, OFFT1, Tref
    i2c.writeto(0x77, b'\x1E')
    time.sleep_ms(20)
    C1 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA2, 2))[0]
    C2 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA4, 2))[0]
    C3 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA6, 2))[0]
    C4 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA8, 2))[0]
    C5 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xAA, 2))[0]
    C6 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xAC, 2))[0]
    SENST1 =  C1*0x8000
    OFFT1 = C2*0x10000
    Tref = C5*0x100
    
setupms5611()
print(C3, C4, C6, SENST1, OFFT1, Tref)

29313 26259 28282 1556152320 3160997888 8063232


In [44]:
def MS5611prepD(d):
    i2c.writeto(0x77, d)
    
def MS5611fetchD():
    r = i2c.readfrom_mem(0x77, 0x00, 3)
    return r[0]*0x10000 + r[1]*0x100 + r[2]

def MS5611readD():
    MS5611prepD(b'\x48')
    time.sleep_ms(20)
    D1 = MS5611fetchD()
    
    MS5611prepD(b'\x58')
    time.sleep_ms(20)
    D2 = MS5611fetchD()
    return D1, D2

def MS5611convert(D1, D2):
    dT = D2 - Tref
    TEMP = 2000 + dT*C6/0x00800000 
    OFF = OFFT1 + dT*C4/0x80 
    SENS = SENST1 + dT*C3 / 0x100
    if TEMP < 2000:
        T2 = dT*dT/0x80000000; 
        ra = TEMP - 2000; 
        ra = ra*ra; 
        OFF -= 5*ra/2; 
        SENS -= 5*ra / 4; 
        if TEMP < -1500:
            rb = TEMP - (-1500) 
            rb = rb*rb 
            OFF -= 7*rb 
            SENS -= 11*rb / 2 
        TEMP -= T2; 
    return (SENS * D1 / 0x200000 - OFF) / 0x8000 
 

In [45]:
print(MS5611readD())

(8746934, 8058206)


In [50]:
D1, D2 = MS5611readD()
print(MS5611convert(D1, D2)-99000)



2567.805


In [54]:
%%writefile ms5611_funcs.py

from i2cmodule import i2c

import ustruct, time

C3, C4, C6, SENST1, OFFT1, Tref = 0,0,0,0,0,0
def setupms5611():
    global C3, C4, C6, SENST1, OFFT1, Tref
    i2c.writeto(0x77, b'\x1E')
    time.sleep_ms(20)
    C1 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA2, 2))[0]
    C2 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA4, 2))[0]
    C3 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA6, 2))[0]
    C4 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA8, 2))[0]
    C5 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xAA, 2))[0]
    C6 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xAC, 2))[0]
    SENST1 =  C1*0x8000
    OFFT1 = C2*0x10000
    Tref = C5*0x100
    
def MS5611readD():
    i2c.writeto(0x77, b'\x48')
    time.sleep_ms(20)
    r = i2c.readfrom_mem(0x77, 0x00, 3)
    D1 = r[0]*0x10000 + r[1]*0x100 + r[2]
    
    i2c.writeto(0x77, b'\x58')
    time.sleep_ms(20)
    r = i2c.readfrom_mem(0x77, 0x00, 3)
    D2 = r[0]*0x10000 + r[1]*0x100 + r[2]
    return D1, D2

def MS5611convert(D1, D2):
    dT = D2 - Tref
    TEMP = 2000 + dT*C6/0x00800000 
    OFF = OFFT1 + dT*C4/0x80 
    SENS = SENST1 + dT*C3 / 0x100
    if TEMP < 2000:
        T2 = dT*dT/0x80000000; 
        ra = TEMP - 2000; 
        ra = ra*ra; 
        OFF -= 5*ra/2; 
        SENS -= 5*ra / 4; 
        if TEMP < -1500:
            rb = TEMP - (-1500) 
            rb = rb*rb 
            OFF -= 7*rb 
            SENS -= 11*rb / 2 
        TEMP -= T2; 
    return (SENS * D1 / 0x200000 - OFF) / 0x8000 


# takes the library module as an argument to save importing it here
async def aMS5611read(uasyncio):
    i2c.writeto(0x77, b'\x48')
    await uasyncio.sleep_ms(20)
    r = i2c.readfrom_mem(0x77, 0x00, 3)
    D1 = r[0]*0x10000 + r[1]*0x100 + r[2]
    
    i2c.writeto(0x77, b'\x58')
    await uasyncio.sleep_ms(20)
    r = i2c.readfrom_mem(0x77, 0x00, 3)
    D2 = r[0]*0x10000 + r[1]*0x100 + r[2]
    
    return MS5611convert(D1, D2)



Writing ms5611_funcs.py



In [22]:
%sendtofile --source ms5611_funcs.py

Sent 49 lines (1497 bytes).


In [24]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.9.2-443-g236297f4 on 2017-11-06; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft reboot\r\nOSError: [Errno 2] ENOENT\r\nMicroPython v1.9.2-443-g236297f4 on 2017-11-06; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [25]:
import ms5611_funcs

In [26]:
ms5611_funcs.setupms5611()

In [31]:
D1, D2 = ms5611_funcs.MS5611readD()
print(ms5611_funcs.MS5611convert(D1, D2))



101606.1


In [53]:
import os
def f(i):
    print(i.listdir())
f(os)


['boot.py', 'aaa', 'bbb.txt', 'ddd', 'dd', 'hhh.txt', 'i2cmodule.py', 'ms5611_funcs.py', 'connecthotspot.py', 'connectwifi.py', 'wificodes.txt', 'lib', 'webserve_funcs.py', 'websocket_funcs.py', 'graphtest.html', 'vl6180_funcs.py', 'raphael.min.js', 'main.py']


In [ ]:
%%writefile ms5611_async.py

import uasyncio
from ms5611_funcs import MS5611prepD, MS5611fetchD, MS5611convert

async def aMS5611read(uasyncio):
    MS5611prepD(b'\x48')
    await uasyncio.sleep_ms(20)
    D1 = MS5611fetchD()
    
    MS5611prepD(b'\x58')
    await uasyncio.sleep_ms(20)
    D2 = MS5611fetchD()
    
    return MS5611convert(D1, D2)

